# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import struct
import time
from decimal import Decimal

# Get weight values

In [2]:
total_num_of_weight = 401
def get_weights_arr():
    weights_arr = []
    w_val = Decimal('-2.0')
    step = Decimal('0.01')
    for i in range (total_num_of_weight):
        weights_arr.append(float(w_val))
        w_val += step
    return weights_arr

weights_arr = get_weights_arr()
#the weight value to be recovered
true_weight = 1.43

# Function Definitions

In [3]:
def float_to_binary_str(f):
    # Pack the float into 4 bytes (32-bit) using IEEE 754 standard
    [packed] = struct.unpack('!I', struct.pack('!f', f))
    # Convert the packed number to a binary string
    return f"{packed:032b}"

In [4]:
def HW_float32(f):
    # Get the binary representation of the 32-bit float
    binary_str = float_to_binary_str(f)
    # Count and return the number of '1' bits
    return binary_str.count('1')

In [5]:
#get one part of the binary representation
def getbyte(f,byte_position):
    if byte_position == 0:#sign bit
        inbinary = float_to_binary_str(f)[0]
    elif byte_position == 4:#last 7 bits
        inbinary = float_to_binary_str(f)[25:32]
    else:
        inbinary = float_to_binary_str(f)[(byte_position-1)*8+1:byte_position*8+1]
    
    return int(inbinary,2)

#### The values to be recovered

In [6]:
#true values of different parts of the weight
true_sign_bit = getbyte(true_weight,0)
true_exponent_bits = getbyte(true_weight,1)
true_byte_two = getbyte(true_weight,2)
true_byte_three = getbyte(true_weight,3)
true_byte_four = getbyte(true_weight,4)
true_value_index = weights_arr.index(true_weight)

In [7]:
print("sign bit = " + str(true_sign_bit) + ", exponent = " + str(true_exponent_bits) + ", first mantissa byte = " + str(true_byte_two) + ", second mantissa byte = " + str(true_byte_three) + ", last 7 bits = " + str(true_byte_four))

sign bit = 0, exponent = 127, first mantissa byte = 110, second mantissa byte = 20, last 7 bits = 61


In [8]:
def get_hypothetical_leakages(num_of_traces, inputs_arr):
    hypothetical_leakages = [[] for i in range(total_num_of_weight)]
    for weight_index in range(total_num_of_weight):
        for j in range(num_of_traces):
            #compute the hypothetical product values for each hypothetical value of weight and each random input
            hypothetical_product = weights_arr[weight_index] * inputs_arr[j]
            #compute hypothetical leakages
            hypothetical_leakages[weight_index].append(HW_float32(hypothetical_product))
    return np.array(hypothetical_leakages)

In [9]:
def CPA_attack(num_of_traces, total_time_sample, start_time_sample, trace_arr, hypothetical_leakages):
    r_abs = [[] for i in range(total_num_of_weight)]
    for t in range(total_time_sample):
        time_sample_index = t + start_time_sample
        for weight_index in range(total_num_of_weight):
            corr_coef = np.corrcoef(hypothetical_leakages[weight_index],trace_arr[:,time_sample_index])[0][1]
            r_abs[weight_index].append(abs(corr_coef))
    return r_abs  

In [10]:
def recover_byte(correlation_arr, byte_position, total_time_sample):
    num_of_different_values = 256 #for byte two and three and exponent bits, we have 256 different values
    if byte_position == 0:#for sign bit, there are just 2 different values
        num_of_different_values = 2
    if byte_position == 4:#for last 7 bits, there are 2^7 different values
        num_of_different_values = 128
    correlations = [[0 for t in range(total_time_sample)] for i in range(num_of_different_values)]
    for weight_index in range(total_num_of_weight):#for each weight
        byte_value = getbyte(weights_arr[weight_index],byte_position)#for the chosen byte of this weight value
        for t in range(total_time_sample):#take the correlations for this weight value, for each time sample, update the correlations
            if correlation_arr[weight_index][t] > correlations[byte_value][t]:
                correlations[byte_value][t] = correlation_arr[weight_index][t]
        
    return correlations

In [11]:
def plot_correlations(correlation, value_index, total_time_sample, start_time_sample):
    x = []
    total_plots = len(correlation)
    
    for t in range(total_time_sample):
        x.append(t+start_time_sample)
    
    plt.plot(x,correlation[value_index], color="r")
    
    for i in range(total_plots):
        if i == value_index:
            continue
        else: 
            plt.plot(x,correlation[i], color="#808080")
    return

In [12]:
def savefile(foldername, filename, arr, total_time_sample, start_time_sample):
    f = open(foldername + "//" + filename + ".txt", "w")
    length = len(arr)#number of columns
    
    
    #first line
    f.write("x ")
    for i in range(length):
        f.write("y"+str(i)+" ")
    f.write("\n")
    
    for t in range(total_time_sample):#for each line
        f.write(str(t+start_time_sample)+" ")#first write down the time sample
        for j in range(length):
            f.write(str(arr[j][t])+" ")
        f.write("\n")
    f.close()
    return

In [13]:
def load_traces(num_of_traces, folder_name):
    trace_waves_arr = []
    inputs_arr = []

    for i in range(num_of_traces):
        with open(folder_name + '/trace_'+str(i)+'.txt') as f:
            lines = f.read().splitlines()
            trace_waves_arr.append(lines)

    with open(folder_name + '/inputs.txt') as f:
        inputs_arr = f.read().splitlines()
    trace_waves_arr = np.array(trace_waves_arr)
    trace_waves_arr = trace_waves_arr.astype(float)

    inputs_arr = np.array(inputs_arr)
    inputs_arr = inputs_arr.astype(float)
    return trace_waves_arr, inputs_arr

In [14]:
#### save the plot raw data
def saveplot(foldername, filename, plot_data, arr_num_traces):                
    #save the file
    f = open(foldername + "//" + filename + ".txt", "w")    
    length = len(arr_num_traces)
    
    #first line---
    f.write("x y\n")
    #other lines
    for i in range(length):#for each line
        f.write(str(arr_num_traces[i])+" " + str(plot_data[i])+"\n")
    f.close()
    return

In [15]:
def find_max_and_max_true(correlation_arr, true_value_index):
    overall_max = 0
    true_max = 0
    total_plots = len(correlation_arr)
    
    for i in range(total_plots):
        if i == true_value_index:
            true_max = max(correlation_arr[i])
        else:
            w_max = max(correlation_arr[i])
            if w_max > overall_max:
                overall_max = w_max
                
    return overall_max, true_max

In [53]:
def run_CPA_n_traces(total_ts, start_ts, trace_arr, save_folder, filename, arr_num_traces, hypothetical_leakages):
    # initialize np arrays for highest correlation results
    arr_max_sign_bit = np.array([])
    arr_max_true_sign_bit = np.array([])
    arr_max_exponent_bits = np.array([])
    arr_max_true_exponent_bits = np.array([])
    arr_max_byte_two = np.array([])
    arr_max_true_byte_two = np.array([])
    arr_max_byte_three = np.array([])
    arr_max_true_byte_three = np.array([])
    arr_max_byte_four = np.array([])
    arr_max_true_byte_four = np.array([])

    total_experiment = len(arr_num_traces)

    start_time = time.time()
    for i in range(total_experiment):
        trace_no = arr_num_traces[i]
        print(f'Running CPA for num_of_traces = {trace_no}')
        
        ind = [j for j in range(trace_no)]
        
        traces = trace_arr[ind]
        hypoleak = hypothetical_leakages[:,ind]#extract the corresponding hypothetical leakages
        r = CPA_attack(num_of_traces=trace_no, total_time_sample=total_ts, start_time_sample=start_ts, trace_arr=traces, hypothetical_leakages=hypoleak)

        print(f'CPA attack for num_of_traces = {trace_no} and ({start}-{start + total_ts}) time samples complete in {time.time() - start_time}!')
        correlations_sign_bit = recover_byte(r, 0, total_ts)
        correlations_exponent_bits = recover_byte(r,1, total_ts)
        correlations_byte_two = recover_byte(r, 2, total_ts)
        correlations_byte_three = recover_byte(r, 3, total_ts)
        correlations_byte_four = recover_byte(r, 4, total_ts)

        max_sign_bit, max_true_sign_bit = find_max_and_max_true(correlations_sign_bit, true_sign_bit)
        max_exponent_bits, max_true_exponent_bits = find_max_and_max_true(correlations_exponent_bits, true_exponent_bits)
        max_byte_two, max_true_byte_two = find_max_and_max_true(correlations_byte_two, true_byte_two)
        max_byte_three, max_true_byte_three = find_max_and_max_true(correlations_byte_three, true_byte_three)
        max_byte_four, max_true_byte_four = find_max_and_max_true(correlations_byte_four, true_byte_four)

        arr_max_sign_bit = np.append(arr_max_sign_bit, max_sign_bit)
        arr_max_true_sign_bit = np.append(arr_max_true_sign_bit, max_true_sign_bit)
        arr_max_exponent_bits = np.append(arr_max_exponent_bits, max_exponent_bits)
        arr_max_true_exponent_bits = np.append(arr_max_true_exponent_bits, max_true_exponent_bits)
        arr_max_byte_two = np.append(arr_max_byte_two, max_byte_two)
        arr_max_true_byte_two = np.append(arr_max_true_byte_two, max_true_byte_two)
        arr_max_byte_three = np.append(arr_max_byte_three, max_byte_three)
        arr_max_true_byte_three = np.append(arr_max_true_byte_three, max_true_byte_three)
        arr_max_byte_four = np.append(arr_max_byte_four, max_byte_four)
        arr_max_true_byte_four = np.append(arr_max_true_byte_four, max_true_byte_four)

        if i%100000 == 0:
            saveplot(save_folder, filename + "sign_bit", arr_max_sign_bit, arr_num_traces)
            saveplot(save_folder, filename + "true_sign_bit", arr_max_true_sign_bit, arr_num_traces)
            saveplot(save_folder, filename + "exponent_bits", arr_max_exponent_bits, arr_num_traces)
            saveplot(save_folder, filename + "true_exponent_bits", arr_max_true_exponent_bits, arr_num_traces)
            saveplot(save_folder, filename + "first_byte", arr_max_byte_two, arr_num_traces)
            saveplot(save_folder, filename + "true_first_byte", arr_max_true_byte_two, arr_num_traces)
            saveplot(save_folder, filename + "second_byte", arr_max_byte_three, arr_num_traces)
            saveplot(save_folder, filename + "true_second_byte", arr_max_true_byte_three, arr_num_traces)
            saveplot(save_folder, filename + "last_bits", arr_max_byte_four, arr_num_traces)
            saveplot(save_folder, filename + "true_last_bits", arr_max_true_byte_four, arr_num_traces)
            
    saveplot(save_folder, filename + "sign_bit", arr_max_sign_bit, arr_num_traces)
    saveplot(save_folder, filename + "true_sign_bit", arr_max_true_sign_bit, arr_num_traces)
    saveplot(save_folder, filename + "exponent_bits", arr_max_exponent_bits, arr_num_traces)
    saveplot(save_folder, filename + "true_exponent_bits", arr_max_true_exponent_bits, arr_num_traces)
    saveplot(save_folder, filename + "first_byte", arr_max_byte_two, arr_num_traces)
    saveplot(save_folder, filename + "true_first_byte", arr_max_true_byte_two, arr_num_traces)
    saveplot(save_folder, filename + "second_byte", arr_max_byte_three, arr_num_traces)
    saveplot(save_folder, filename + "true_second_byte", arr_max_true_byte_three, arr_num_traces)
    saveplot(save_folder, filename + "last_bits", arr_max_byte_four, arr_num_traces)
    saveplot(save_folder, filename + "true_last_bits", arr_max_true_byte_four, arr_num_traces)
    
    return

# Attack on unprotected traces

In [17]:
traces, inputs = load_traces(num_of_traces=5000, folder_name="unprotected")
hypothetical_leakages = get_hypothetical_leakages(num_of_traces=5000, inputs_arr=inputs)

In [44]:
start = 490
end = 1011
total = end - start

In [51]:
#unprotected case
num_trace = []
for n in range(3,21):
    num_trace.append(n)
for n in range(40,5020,20):
    num_trace.append(n)

In [52]:
run_CPA_n_traces(total_ts=total, start_ts=start, trace_arr=traces, save_folder="plots_unprotected", filename="", arr_num_traces=num_trace, hypothetical_leakages=hypothetical_leakages)

Running CPA for num_of_traces = 3
CPA attack for num_of_traces = 3 and (490-1011) time samples complete in 8.785578966140747!
Running CPA for num_of_traces = 4
CPA attack for num_of_traces = 4 and (490-1011) time samples complete in 17.639158010482788!
Running CPA for num_of_traces = 5
CPA attack for num_of_traces = 5 and (490-1011) time samples complete in 26.514605045318604!
Running CPA for num_of_traces = 6
CPA attack for num_of_traces = 6 and (490-1011) time samples complete in 35.40126299858093!
Running CPA for num_of_traces = 7
CPA attack for num_of_traces = 7 and (490-1011) time samples complete in 44.26145887374878!
Running CPA for num_of_traces = 8
CPA attack for num_of_traces = 8 and (490-1011) time samples complete in 53.101141929626465!
Running CPA for num_of_traces = 9
CPA attack for num_of_traces = 9 and (490-1011) time samples complete in 62.1882438659668!
Running CPA for num_of_traces = 10
CPA attack for num_of_traces = 10 and (490-1011) time samples complete in 71.9477

# Attack on protected traces

In [ ]:
folder_name="protected"

In [ ]:
def get_inputs(folder_name):
    with open(folder_name + '/inputs.txt') as f:
        inputs_arr = f.read().splitlines()
    f.close()
    inputs_arr = np.array(inputs_arr)
    inputs_arr = inputs_arr.astype(float)
    return inputs_arr

In [ ]:
inputs_pro = get_inputs(folder_name)

In [ ]:
hypothetical_leakages = get_hypothetical_leakages(num_of_traces=1000000, inputs_arr=inputs_pro)

In [ ]:
#protected case
num_trace = []
for n in range(3,99,3):
    num_trace.append(n)

for n in range(100, 1000000, 10000):
    num_trace.append(n)
num_trace.append(999999)
len(num_trace)

In [ ]:
def compute_correlation(time_sample, num_trace, hypothetical_leakages):
    #get all data for the time_sample
    trace = []
    #total number of experiments to run
    number_of_experiments = len(num_trace)
    #all correlations, for each number of traces and for each weight value
    r_abs = [[] for i in range(total_num_of_weight)]

    for i in range(num_of_traces):
        with open(folder_name + '/trace_'+str(i)+'.txt') as f:
            lines = f.read().splitlines()
            trace.append(lines[time_sample])
            f.close()
    
    trace = np.array(trace)
    trace = trace.astype(float)
    
    print("got traces")
    for trace_no in num_trace:
        
        ind = [j for j in range(trace_no)]
        
        trace_subset = trace[ind]
        hypoleak_subset = hypothetical_leakages[:,ind]#extract the corresponding hypothetical leakages
        
        for weight_index in range(total_num_of_weight):
            corr_coef = np.corrcoef(hypoleak_subset[weight_index],trace_subset)[0][1]
            r_abs[weight_index].append(abs(corr_coef))
    
    return r_abs 

In [ ]:
start = 490
end_pro = 4301
total_pro = end_pro - start

In [67]:
def CPA_protected(hypothetical_leakages, num_trace):
    total_experiment = len(num_trace)
    r_all = [[[0 for k in range(total_pro)] for i in range(total_num_of_weight)]for j in range(total_experiment)]
    for i in range(total_pro):
        time_sample = i + start
        print("computing for time sample " + str(time_sample))
        r_t = compute_correlation(time_sample, num_trace, hypothetical_leakages)
        for j in range(total_experiment):
            for weight_index in range(total_num_of_weight):
                r_all[j][weight_index][i] = r_t[weight_index][j]
    return r_all

133

In [ ]:
def run_CPA_n_traces(total_ts, start_ts, trace_arr, save_folder, filename, arr_num_traces, hypothetical_leakages):
    # initialize np arrays for highest correlation results
    arr_max_sign_bit = np.array([])
    arr_max_true_sign_bit = np.array([])
    arr_max_exponent_bits = np.array([])
    arr_max_true_exponent_bits = np.array([])
    arr_max_byte_two = np.array([])
    arr_max_true_byte_two = np.array([])
    arr_max_byte_three = np.array([])
    arr_max_true_byte_three = np.array([])
    arr_max_byte_four = np.array([])
    arr_max_true_byte_four = np.array([])

    total_experiment = len(arr_num_traces)
    r_all = CPA_protected(hypothetical_leakages=hypothetical_leakages, num_trace=arr_num_traces)

    for i in range(total_experiment):        
        

        r = r_all[i]

        correlations_sign_bit = recover_byte(r, 0, total_ts)
        correlations_exponent_bits = recover_byte(r,1, total_ts)
        correlations_byte_two = recover_byte(r, 2, total_ts)
        correlations_byte_three = recover_byte(r, 3, total_ts)
        correlations_byte_four = recover_byte(r, 4, total_ts)

        max_sign_bit, max_true_sign_bit = find_max_and_max_true(correlations_sign_bit, true_sign_bit)
        max_exponent_bits, max_true_exponent_bits = find_max_and_max_true(correlations_exponent_bits, true_exponent_bits)
        max_byte_two, max_true_byte_two = find_max_and_max_true(correlations_byte_two, true_byte_two)
        max_byte_three, max_true_byte_three = find_max_and_max_true(correlations_byte_three, true_byte_three)
        max_byte_four, max_true_byte_four = find_max_and_max_true(correlations_byte_four, true_byte_four)

        arr_max_sign_bit = np.append(arr_max_sign_bit, max_sign_bit)
        arr_max_true_sign_bit = np.append(arr_max_true_sign_bit, max_true_sign_bit)
        arr_max_exponent_bits = np.append(arr_max_exponent_bits, max_exponent_bits)
        arr_max_true_exponent_bits = np.append(arr_max_true_exponent_bits, max_true_exponent_bits)
        arr_max_byte_two = np.append(arr_max_byte_two, max_byte_two)
        arr_max_true_byte_two = np.append(arr_max_true_byte_two, max_true_byte_two)
        arr_max_byte_three = np.append(arr_max_byte_three, max_byte_three)
        arr_max_true_byte_three = np.append(arr_max_true_byte_three, max_true_byte_three)
        arr_max_byte_four = np.append(arr_max_byte_four, max_byte_four)
        arr_max_true_byte_four = np.append(arr_max_true_byte_four, max_true_byte_four)

        if i%100000 == 0:
            saveplot(save_folder, filename + "sign_bit", arr_max_sign_bit, arr_num_traces)
            saveplot(save_folder, filename + "true_sign_bit", arr_max_true_sign_bit, arr_num_traces)
            saveplot(save_folder, filename + "exponent_bits", arr_max_exponent_bits, arr_num_traces)
            saveplot(save_folder, filename + "true_exponent_bits", arr_max_true_exponent_bits, arr_num_traces)
            saveplot(save_folder, filename + "first_byte", arr_max_byte_two, arr_num_traces)
            saveplot(save_folder, filename + "true_first_byte", arr_max_true_byte_two, arr_num_traces)
            saveplot(save_folder, filename + "second_byte", arr_max_byte_three, arr_num_traces)
            saveplot(save_folder, filename + "true_second_byte", arr_max_true_byte_three, arr_num_traces)
            saveplot(save_folder, filename + "last_bits", arr_max_byte_four, arr_num_traces)
            saveplot(save_folder, filename + "true_last_bits", arr_max_true_byte_four, arr_num_traces)
            
    saveplot(save_folder, filename + "sign_bit", arr_max_sign_bit, arr_num_traces)
    saveplot(save_folder, filename + "true_sign_bit", arr_max_true_sign_bit, arr_num_traces)
    saveplot(save_folder, filename + "exponent_bits", arr_max_exponent_bits, arr_num_traces)
    saveplot(save_folder, filename + "true_exponent_bits", arr_max_true_exponent_bits, arr_num_traces)
    saveplot(save_folder, filename + "first_byte", arr_max_byte_two, arr_num_traces)
    saveplot(save_folder, filename + "true_first_byte", arr_max_true_byte_two, arr_num_traces)
    saveplot(save_folder, filename + "second_byte", arr_max_byte_three, arr_num_traces)
    saveplot(save_folder, filename + "true_second_byte", arr_max_true_byte_three, arr_num_traces)
    saveplot(save_folder, filename + "last_bits", arr_max_byte_four, arr_num_traces)
    saveplot(save_folder, filename + "true_last_bits", arr_max_true_byte_four, arr_num_traces)
    
    return

In [ ]:
def compute_correlation(time_sample, num_trace, hypothetical_leakages):

    number_of_experiments = len(num_trace)
    #all correlations, for each number of traces and for each weight value
    r_abs = np.zeros((number_of_experiments, total_num_of_weight))


    # Efficiently read only the required line from each trace file
    def read_time_sample(i):
        with open(f"{folder_name}/trace_{i}.txt") as f:
            for _ in range(time_sample):  # Skip lines up to time_sample
                next(f)
            return float(f.readline().strip())  # Read the required line

    # Read all traces efficiently
    traces = np.fromiter((read_time_sample(i) for i in range(num_of_traces)), dtype=np.float64)

    
    for index in range(number_of_experiments):
        trace_no = num_trace[index]
        
        trace_subset = traces[:trace_no]  # Select the first 'trace_no' traces
        hypoleak_subset = hypothetical_leakages[:, :trace_no]  # Select corresponding hypothetical leakages
        
        # Compute correlation efficiently using vectorized NumPy operations
        mean_trace = np.mean(trace_subset)
        mean_hypo = np.mean(hypoleak_subset, axis=1)

        num = np.sum((hypoleak_subset - mean_hypo[:, None]) * (trace_subset - mean_trace), axis=1)
        den = np.sqrt(np.sum((hypoleak_subset - mean_hypo[:, None])**2, axis=1) * np.sum((trace_subset - mean_trace)**2))

        r_abs[index, :] = np.abs(num / den)  # Compute absolute correlation


            
    return r_abs


def CPA_protected(hypothetical_leakages, num_trace):
    total_experiment = len(num_trace)
    
    # Initialize r_all as a NumPy array instead of a nested list
    r_all = np.zeros((total_experiment, total_num_of_weight, total_pro))
    
    for i in range(total_pro):
        time_sample = i + start
        if i%100 == 0:
            print(f"computing for time sample {time_sample}")
        
        # Compute correlation
        r_t = compute_correlation(time_sample, num_trace, hypothetical_leakages)  # Assuming r_t has shape (total_experiment, total_num_of_weight)
        
        # Use NumPy slicing instead of nested loops
        r_all[:, :, i] = r_t  # Assign entire 2D slice in one step

    return r_all

In [ ]:
def CPA_protected(num_trace, hypothetical_leakages):
    total_experiment = len(num_trace)

    # Initialize r_all using NumPy
    r_all = np.zeros((total_experiment, total_num_of_weight, total_pro))
    
    # Function to read a specific line from trace files efficiently
    def read_time_sample(i, time_sample):
        with open(f"{folder_name}/trace_{i}.txt") as f:
            for _ in range(time_sample):  # Skip lines up to time_sample
                next(f)
            return float(f.readline().strip())  # Read the required line    

    for i in range(total_pro):
        time_sample = i + start
        if i % 100 == 0:
            print(f"Computing for time sample {time_sample}")

        # Read only necessary time samples from traces
        traces = np.fromiter((read_time_sample(i, time_sample) for i in range(max(num_trace))), dtype=np.float64)

        # Compute correlation for each experiment
        for index, trace_no in enumerate(num_trace):
            trace_subset = traces[:trace_no]  # Select relevant traces
            hypoleak_subset = hypothetical_leakages[:, :trace_no]  # Select corresponding hypothetical leakages

            # Compute correlation efficiently
            mean_trace = np.mean(trace_subset)
            mean_hypo = np.mean(hypoleak_subset, axis=1)

            num = np.sum((hypoleak_subset - mean_hypo[:, None]) * (trace_subset - mean_trace), axis=1)
            den = np.sqrt(np.sum((hypoleak_subset - mean_hypo[:, None])**2, axis=1) * np.sum((trace_subset - mean_trace)**2))

            r_all[index, :, i] = np.abs(num / den)  # Store absolute correlation

    return r_all


In [ ]:
def CPA_protected(num_trace, hypothetical_leakages):
    total_experiment = len(num_trace)

    # Initialize r_all using NumPy
    r_all = np.zeros((total_experiment, total_num_of_weight, total_pro))

    # Function to read a specific line from trace files efficiently
    def read_time_sample(i, time_sample):
        with open(f"{folder_name}/trace_{i}.txt") as f:
            for _ in range(time_sample):  # Skip lines up to time_sample
                next(f)
            return float(f.readline().strip())  # Read the required line    

    # Precompute statistics for hypothetical leakages
    precomputed_means = []
    precomputed_variances = []
    
    for index, trace_no in enumerate(num_trace):
        hypoleak_subset = hypothetical_leakages[:, :trace_no]  # Select corresponding hypothetical leakages

        mean_hypo = np.mean(hypoleak_subset, axis=1)  # Compute mean once
        var_hypo = np.sum((hypoleak_subset - mean_hypo[:, None])**2, axis=1)  # Compute variance sum once

        precomputed_means.append(mean_hypo)
        precomputed_variances.append(var_hypo)

    # Convert to NumPy arrays for faster indexing
    precomputed_means = np.array(precomputed_means)
    precomputed_variances = np.array(precomputed_variances)

    for i in range(total_pro):
        time_sample = i + start
        if i % 100 == 0:
            print(f"Computing for time sample {time_sample}")

        # Read only necessary time samples from traces
        traces = np.fromiter((read_time_sample(i, time_sample) for i in range(max(num_trace))), dtype=np.float64)

        for index, trace_no in enumerate(num_trace):
            trace_subset = traces[:trace_no]  # Select relevant traces
            
            mean_trace = np.mean(trace_subset)
            trace_dev = trace_subset - mean_trace
            var_trace = np.sum(trace_dev**2)

            num = np.sum((hypothetical_leakages[:, :trace_no] - precomputed_means[index][:, None]) * trace_dev, axis=1)
            den = np.sqrt(precomputed_variances[index] * var_trace)

            r_all[index, :, i] = np.abs(num / den)  # Store absolute correlation

    return r_all
